In [24]:
import glob
import numpy as np
import pandas as pd

### Generate input files

possible basis sets  : 631G*

possible functionals : B3LYP

In [25]:
def load_xyz(filename):
    with open(filename, 'r') as content:
        xyz = content.readlines()[2:]
        #xyz = [_.strip('\n') for _ in xyz]
    return xyz

def opt_input(filename, xyz, **kwargs):
    charge = 0
    mult   = 1

    s = f'$molecule\n{charge} {mult}\n'
    for line in xyz:
        s += f' {line}'
    s += '$end\n\n'
    s += '$rem\n'
    s += f' BASIS = 6-31G*\n'
    s += f' GUI = 2\n'
    s += f' JOBTYPE = Optimization\n'
    s += f' METHOD = B3LYP\n'
    s += f' MOLDEN_FORMAT false\n'
    s += f'$end'
    
    with open(filename, 'w') as f:
        f.write(s)
    f.close()
    

def spe_input(filename, xyz, **kwargs):
    charge, mult = 0, 1
    s = f'$molecule\n{charge} {mult}\n'
    for line in xyz:
        s += f' {line}'
    s += '$end\n\n'
    s += '$rem\n'
    s += f' BASIS = 6-31G*\n'
    s += f' GUI = 2\n'
    s += f' METHOD = B3LYP\n'
    s += f' PRINT_ORBITALS false\n'
    s += f' MOLDEN_FORMAT false\n'
    s += f'$end'
    
    with open(filename, 'w') as f:
        f.write(s)
    f.close()
    
def tddft_input(filename, xyz, **kwargs):
    charge, mult = 0, 1
    s = f'$molecule\n{charge} {mult}\n'
    for line in xyz:
        s += f' {line}'
    s += '$end\n\n'
    s += '$rem\n'
    s += f' BASIS = 6-31G*\n'
    s += f' GUI = 2\n'
    s += f' METHOD = B3LYP\n'
    s += f' PRINT_ORBITALS false\n'
    s += f' MOLDEN_FORMAT false\n'
    s += f'$end'
    
    with open(filename, 'w') as f:
        f.write(s)
    f.close()
    
    
# def generate_input_qchem(filename, xyz, charge, mult, omega,
#                          functional = 'LRC-wPBE', basis_set = 'def2-SVP',
#                          **kwargs):
#     omega_fmt = int(omega*1e3)
#     s = f'$molecule\n{charge} {mult}\n'
#     for line in xyz:
#         s += f' {line}'
#     s += '$end\n\n'
#     s += '$rem\n'
#     s += f' EXCHANGE  {functional}\n'
#     s += f' BASIS  {basis_set}\n'
#     s += f' LRC_DFT TRUE\n'
#     s += f' OMEGA {omega_fmt}\n'
#     s += f' PRINT_ORBITALS false\n'
#     s += f' MOLDEN_FORMAT false\n'
#     s += f'$end'

#     with open(filename, 'w') as f:
#         f.write(s)
#     f.close()


In [19]:
filename = 'fragments/A001/conf-0/xtbopt.xyz'

In [20]:
xyz = load_xyz(filename)
spe_input('fragments/A001/conf-0/spe_input.inp', xyz)
tddft_input('fragments/A001/conf-0/tddft_input.inp', xyz)

In [26]:
mol_dirs = glob.glob('fragments/*/conf-*/')
# conf_dirs = mol_dirs.copy()
len(mol_dirs)

889

In [32]:
for dir_ in mol_dirs:
    xyz = load_xyz(f'{dir_}xtbopt.xyz')
    spe_input(f'{dir_}spe_input.inp', xyz)
    tddft_input(f'{dir_}tddft_input.inp', xyz)
    


In [28]:
# get the job lists
spe_jobs = []
tddft_jobs = []
for mol_dir in mol_dirs:
    spe_jobs.append(f'{mol_dir}spe_input.inp')
    tddft_jobs.append(f'{mol_dir}tddft_input.inp')

In [38]:
spe_jobs[0][:-13]
tddft_jobs[0][:-15]

'fragments/C085/conf-7/'

In [39]:
with open('spe_joblist.txt', 'w') as f:
    for job in spe_jobs:
        f.write(f'{job[:-13]}\n')
    
with open('tddft_joblist.txt', 'w') as f:
    for job in tddft_jobs:
        f.write(f'{job[:-5]}\n')

### generate inputs for default lc-wpbe

In [9]:
jobs = []
for mol_id, omega in zip(opt_omegas['mol_id'], opt_omegas['omega']): #mol_ids
    dir_ = f'mols_crest_done/{mol_id}/'    
    confs = glob.glob(dir_+'conf-*/')
    
    for conf in confs:
        xyz = load_xyz(f'{conf}/conformer.xyz')
        #conf_id = conf.split('/')[-1].split('_')[-1].split('.')[0]
#        print(conf_id)
        generate_input_qchem(f'{conf}/def_lcwpbe_spe.inp', xyz, 0, 1, 0.4)
        jobs.append(f'{conf}')

In [10]:
with open('def_spe_joblist.txt', 'w') as f:
    for job in jobs:
        f.write(f'{job}\n')

In [37]:
# generate the geometry optimization inputs
jobs = []
functionals = ['BP86']
for mol_id in mols: #mol_ids
    dir_ = f'mols_crest_done/{mol_id}/'    
    confs = glob.glob(dir_+'conf-*/')
    
    for conf in confs:
        xyz = load_xyz(f'{conf}/conformer.xyz')
        #conf_id = conf.split('/')[-1].split('_')[-1].split('.')[0]
#        print(conf_id)
        for functional in functionals:
            opt_input(f'{conf}/{functional}_opt.inp', xyz)
            jobs.append(f'{conf}')

    

In [14]:
mol_dirs = glob.glob('fragments/*/conf-*/')
# conf_dirs = mol_dirs.copy()
len(mol_dirs)



889

In [39]:
with open('opt_joblist.txt', 'w') as f:
    for job in jobs:
        f.write(f'{job}\n')

### generate the input files for the DFT geometry optimizations from CREST ensenble

In [40]:
# generate the single points
jobs = []
functionals = ['BP86', 'B3LYP', 'PBE0', 'M06-2X']
for mol_id in mols: #mol_ids
    dir_ = f'mols_crest_done/{mol_id}/'    
    confs = glob.glob(dir_+'conf-*/')
    
    for conf in confs:
        xyz = load_xyz(f'{conf}/conformer.xyz')
        #conf_id = conf.split('/')[-1].split('_')[-1].split('.')[0]
#        print(conf_id)
        for functional in functionals:
            spe_input(f'{conf}/{functional}_spe.inp', xyz, functional)
        jobs.append(f'{conf}')


In [42]:
;

1912